In [1]:
import pulp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy

class IP_Solver_Capsule:
    def __init__(self, name="Integer_Programming_Model", sense='max'):
        """
        初始化整数规划求解器
        :param name: 模型名称
        :param sense: 'max' (最大化) 或 'min' (最小化)
        """
        self.name = name
        self.sense = pulp.LpMaximize if sense == 'max' else pulp.LpMinimize
        self.prob = pulp.LpProblem(name, self.sense)
        
        # 内部变量存储
        self.vars = {}           # 扁平化存储所有变量 {'x_0_1': var_obj}
        self.matrix_vars = {}    # 矩阵化存储 {'x': [[var...], ...]}
        self.constraints = {}
        self.objective_expr = None
        
        # 审计元数据 (用于 audit 阶段的逻辑检查)
        self.metadata = {
            'matrix_dims': {},   # 记录矩阵变量的维度 {'x': (rows, cols)}
            'big_m_count': 0,    # Big-M 约束计数
            'var_count': 0
        }
        
        self.is_solved = False
        self.status = None

    def add_variable_matrix(self, row_indices, col_indices, name="x", cat='Binary'):
        """
        [关键] 添加矩阵变量 x_ij (适用于指派问题、TSP、选址)
        :param row_indices: 行索引列表 (如 Workers, Cities)
        :param col_indices: 列索引列表 (如 Tasks, TimeSlots)
        :param cat: 'Binary'(0/1) 或 'Integer'
        """
        # 使用 PuLP 的字典变量功能
        var_dict = pulp.LpVariable.dicts(
            name, 
            (row_indices, col_indices), 
            cat=getattr(pulp, f"Lp{cat}"), 
            lowBound=0
        )
        
        # 注册到内部存储
        self.matrix_vars[name] = var_dict
        self.metadata['matrix_dims'][name] = (len(row_indices), len(col_indices))
        
        # 扁平化映射方便后续检索
        for r in row_indices:
            for c in col_indices:
                self.vars[f"{name}_{r}_{c}"] = var_dict[r][c]
                self.metadata['var_count'] += 1
                
        print(f"✅ 已注册矩阵变量 {name}_ij: 维度 {len(row_indices)}x{len(col_indices)} (类型: {cat})")
        return var_dict

    def add_big_M_constraint(self, bin_var, cont_var, M=100000, relation='<=', name="BigM_Link"):
        """
        [逻辑要求 6] 混合逻辑约束封装 (Big-M Method)
        场景：只有当 0/1 变量 bin_var=1 时，连续变量 cont_var 才有意义 (cont_var <= M * bin_var)
        """
        if relation == '<=':
            expr = (cont_var <= M * bin_var)
        elif relation == '>=':
            expr = (cont_var >= bin_var / M) # 仅示例，视具体逻辑而定
        else:
            raise ValueError("不支持的关系符")
            
        self.prob += expr, name
        self.constraints[name] = expr
        self.metadata['big_m_count'] += 1
        print(f"🔗 添加混合逻辑约束 (Big-M): {name}")

    def add_TSP_subtour_elimination(self, u_vars, x_vars, cities):
        """
        [场景扩展] TSP 问题的子回路消除约束 (MTZ Formulation)
        u_i - u_j + n * x_ij <= n - 1
        """
        n = len(cities)
        count = 0
        for i in cities[1:]: # 排除起点
            for j in cities[1:]:
                if i != j:
                    # MTZ 约束公式
                    self.prob += (u_vars[i] - u_vars[j] + n * x_vars[i][j] <= n - 1), f"Subtour_{i}_{j}"
                    count += 1
        print(f"🚧 已添加 TSP 子回路消除约束: {count} 条")

    def set_objective(self, expr):
        """设置目标函数"""
        self.prob += expr
        self.objective_expr = expr

    def add_constraint(self, expr, sense, rhs, name):
        """通用约束添加"""
        if sense == '<=': c = (expr <= rhs)
        elif sense == '>=': c = (expr >= rhs)
        elif sense == '==': c = (expr == rhs)
        else: raise ValueError("Sense error")
        self.prob += c, name
        self.constraints[name] = c

    def audit(self):
        """
        [核心] 模型审计：执行 6 大逻辑检查
        """
        print("\n🛡️ === 逻辑严谨性审计 (Audit Phase) ===")
        issues = []
        is_safe = True

        # 检查 1: 求解难度陷阱 (数据量极大)
        if self.metadata['var_count'] > 5000: # 阈值可调整
            msg = f"⚠️ [难度预警] 变量数量达 {self.metadata['var_count']}。属于大规模 NP-Hard 问题。" \
                  f"\n   -> 建议: 若求解超时，请在论文中声明改用 '启发式算法(Heuristic)' 如遗传算法或蚁群算法。"
            issues.append(msg)
            # 注意：此处不强制 return False，只是警告

        # 检查 3: 人数与任务不匹配 (指派问题) -> 引入虚拟变量
        for name, dims in self.metadata['matrix_dims'].items():
            rows, cols = dims
            if rows != cols:
                msg = f"⚠️ [不平衡预警] 变量矩阵 '{name}' 维度为 {rows}x{cols} (非方阵)。" \
                      f"\n   -> 风险: 标准指派问题可能无解。" \
                      f"\n   -> 建议: 引入 '虚拟变量(Dummy Variables)' 补齐为方阵 (Cost=0 或 Cost=BigM)。"
                issues.append(msg)

        # 检查 5: 约束完整性 (是否裸奔)
        if not self.constraints:
            issues.append("❌ [逻辑错误] 未检测到任何约束条件。模型将无界或无意义。")
            is_safe = False
        
        if self.objective_expr is None:
            issues.append("❌ [逻辑错误] 未设置目标函数。")
            is_safe = False

        # 检查 6: 混合逻辑提示
        if self.metadata['big_m_count'] > 0:
            print(f"ℹ️ [信息] 检测到 {self.metadata['big_m_count']} 个混合逻辑约束(Big-M)。请确保 M 值足够大但不过大(防精度溢出)。")

        # 输出审计报告
        if not issues:
            print("✅ 审计通过：模型结构逻辑看似严谨。")
            return True
        else:
            print("🚨 审计发现潜在风险：")
            for i in issues: print(i)
            return is_safe

    def solve(self, time_limit=300):
        """求解与结果校验"""
        print("\n🚀 开始求解...")
        # 使用 CBC 求解器，设置时间限制防止死循环
        solver = pulp.PULP_CBC_CMD(msg=1, timeLimit=time_limit)
        status = self.prob.solve(solver)
        self.status = pulp.LpStatus[status]
        self.is_solved = True
        
        print(f"📊 求解状态: {self.status}")
        
        if self.status == 'Optimal':
            obj_val = pulp.value(self.prob.objective)
            print(f"💎 目标函数值: {obj_val}")
            
            # 检查 4: 最大化时结果与实际意义匹配性
            if self.sense == pulp.LpMaximize and obj_val < 0:
                print(f"⚠️ [结果校验] 最大化目标值为负 ({obj_val})。请检查：")
                print("   1. 目标系数是否存在负值？")
                print("   2. 是否遗漏了基准收益？")
                print("   3. 这是否符合物理意义？")
        else:
            print("⚠️ 未找到最优解 (可能是 Infeasible 或 Unbounded)。请检查约束冲突。")

    def analyze_sensitivity(self):
        """
        [加分项] 整数规划的灵敏度分析
        策略：线性松弛 (Linear Relaxation)。
        原理：将整数约束暂时移除，视为 LP 问题，获取对偶值(Shadow Price)。
        """
        if not self.is_solved: return
        print("\n🔍 === 灵敏度分析 (基于线性松弛) ===")
        print("💡 说明: 整数规划无直接影子价格。系统将执行 'Linear Relaxation' 以估算资源边际价值。")
        
        # 1. 创建模型副本
        relaxed_prob = copy.deepcopy(self.prob)
        
        # 2. 将所有变量放宽为连续变量
        for v in relaxed_prob.variables():
            v.cat = pulp.LpContinuous
            
        # 3. 静默求解
        relaxed_prob.solve(pulp.PULP_CBC_CMD(msg=0))
        
        if relaxed_prob.status == 1:
            shadow_data = []
            for name, c in relaxed_prob.constraints.items():
                if abs(c.pi) > 1e-5: # 只记录有价值的约束
                    shadow_data.append({'Constraint': name, 'Marginal_Value': c.pi})
            
            df_shadow = pd.DataFrame(shadow_data).sort_values(by='Marginal_Value', key=abs, ascending=False)
            
            print("🔥 关键资源瓶颈 (Top 5 边际价值):")
            print(df_shadow.head(5).to_string(index=False))
            
            # 简单绘图
            if not df_shadow.empty:
                plt.figure(figsize=(8, 4))
                sns.barplot(data=df_shadow.head(8), x='Marginal_Value', y='Constraint', palette='viridis')
                plt.title('Resource Sensitivity (Linear Relaxation Proxy)')
                plt.show()
        else:
            print("⚠️ 线性松弛求解失败，无法分析灵敏度。")

    def export_results(self):
        """导出结果与 LaTeX"""
        if not self.is_solved: return
        
        # 提取非零变量
        data = []
        for v in self.prob.variables():
            if abs(v.varValue) > 1e-5:
                data.append({'Variable': v.name, 'Value': v.varValue})
        
        df = pd.DataFrame(data)
        df.to_excel(f"{self.name}_Solution.xlsx", index=False)
        print(f"\n💾 结果已保存: {self.name}_Solution.xlsx")
        
        # 打印 LaTeX
        print("\n📝 LaTeX 表格代码 (Top 10):")
        print(df.head(10).to_latex(index=False))

    def get_methodology(self):
        return """
        \\subsection{Integer Programming Formulation}
        We formulated the problem as a 0-1 Integer Programming (IP) model. 
        Decision variables $x_{ij}$ are binary, indicating selection. 
        Constraints include logical assignments and resource limits. 
        For mixed-logic conditions, the Big-M method was employed.
        Sensitivity analysis was conducted via Linear Relaxation to identify critical constraints.
        """

# 美赛 IP 建模分步交互工作流 (Step-by-Step Workflow)

> **核心策略**：这是一个非常明智的调整。在美赛高压环境下，试图用一个巨型 Prompt 一次性生成所有代码通常会失败（变量名对不上、逻辑遗漏、调试困难）。“步步为营” (Step-by-Step) 的策略不仅能降低 AI 的幻觉风险，还能让你在每一步都有机会人工介入检查（Audit）。

我们将工作流拆解为 **5 个独立的交互回合**。你需要准备 5 个对应的提示词模板 (TXT 文件)。

---

## Phase 0: 握手与上下文注入 (Context Injection)

* **时机**：打开 Notebook 后，运行完 `IP_Solver_Capsule` 类定义代码之后。
* **目的**：让 AI 知道它拥有什么工具，以及当前要解决什么问题。

### 📄 模板 0：初始化 (`Prompt_0_Init.txt`)

```markdown
【系统指令：初始化 IP 建模环境】

背景：
1. 我已在环境中运行了 `IP_Solver_Capsule` 类的定义（包含 add_variable_matrix, add_big_M_constraint, audit 等方法）。
2. 我们正在解决一个 [在此填入问题类型，如：多层级物流网络选址] 问题。

任务：
请确认你已理解 `IP_Solver_Capsule` 的用法。不需要输出代码，只需回复“系统就绪”并简述你对该类核心功能的理解。
```

---

## Phase 1: 数据与变量 (Data & Variables)

* **时机**：数据已清洗完毕，准备定义数学符号时。
* **目的**：读取数据，建立索引，声明决策变量。

### 📄 模板 1：变量定义 (`Prompt_1_Variables.txt`)

```markdown
【系统指令：Phase 1 数据读取与变量定义】

请生成 Python 代码执行以下步骤：

1. **数据读取**：
   - 读取文件：[例如：'data_clean.xlsx']
   - 提取索引集合：
     - 集合 I (Rows): [例如：df['Site_ID'].unique()]
     - 集合 J (Cols): [例如：df['Customer_ID'].unique()]
   - 提取参数：
     - [例如：Cost_Matrix = df.set_index(...).to_dict()]
     - [例如：Capacity_Dict = ...]

2. **实例化求解器**：
   - 名称：[例如：'Location_Problem']，方向：[min/max]。

3. **定义变量** (使用 solver.add_variable_matrix 或 add_variable_list)：
   - 变量 1: [例如：x[i][j], Binary, 代表客户 j 指派给站点 i]
   - 变量 2: [例如：y[i], Binary, 代表站点 i 是否建设]
   - 变量 3: [例如：flow[i][j], Continuous, 代表运输量]

要求：输出代码即可，不要解释。确保变量名简短清晰。
```

---

## Phase 2: 约束与审计 (Constraints & Audit)

* **时机**：变量定义完成，需要写入业务逻辑时。这是最关键的一步。
* **目的**：翻译数学约束，并利用 `audit()` 进行逻辑自检。

### 📄 模板 2：约束构建 (`Prompt_2_Constraints.txt`)

```markdown
【系统指令：Phase 2 约束构建与审计】

基于上一轮定义的变量，请生成添加约束的代码：

1. **约束逻辑**：
   - 约束 A (硬约束): [例如：每个 Customer 必须被指派且仅指派给 1 个 Site]
   - 约束 B (资源限制): [例如：Site i 的总服务量 <= Capacity[i]]
   - 约束 C (混合逻辑 Big-M): [例如：只有当 y[i]=1 (建设) 时，x[i][j] 才能 > 0。使用 solver.add_big_M_constraint 方法，M取总需求量。]

2. **执行审计**：
   - 调用 `solver.audit()`。
   - 添加一段注释，提示我检查输出的审计报告（如供需是否平衡）。

要求：使用 `pulp.lpSum` 进行求和。给每个约束起一个有意义的名字 (如 'Demand_Sat_C1')。
```

---

## Phase 3: 目标与求解 (Objective & Solve)

* **时机**：审计通过（Audit Passed）后。
* **目的**：设定目标函数，运行求解器，并进行物理意义检查。

### 📄 模板 3：求解 (`Prompt_3_Solve.txt`)

```markdown
【系统指令：Phase 3 设定目标与求解】

1. **目标函数**：
   - 目标是 [Minimize / Maximize]：
   - 公式组成：[例如：sum(x * Transport_Cost) + sum(y * Build_Cost)]
   - 使用 `solver.set_objective()` 设置。

2. **求解与检查**：
   - 调用 `solver.solve(time_limit=300)`。
   - 编写一个 `if` 判断：如果求解状态是 Optimal，打印目标函数值；如果目标值 < 0 且是最大化问题，打印“物理意义警告”。

要求：输出代码即可。
```

---

## Phase 4 & 5: 灵敏度与交付 (Analysis & Delivery)

* **时机**：得到最优解后。
* **目的**：挖掘加分项（影子价格），生成论文图表。

### 📄 模板 4：分析与交付 (`Prompt_4_Delivery.txt`)

```markdown
【系统指令：Phase 4 & 5 结果分析与交付】

模型已求解成功，请生成后续分析代码：

1. **灵敏度分析 (Phase 4)**：
   - 直接调用 `solver.analyze_sensitivity()` (基于线性松弛)。

2. **结果导出 (Phase 5)**：
   - 调用 `solver.export_results()` 导出 Excel 和 LaTeX。
   - 打印 `solver.get_methodology()` 获取方法论描述。

3. **可视化**：
   - [在此描述具体图表需求]
   - 例如：请提取 x 变量的结果，转为 DataFrame 矩阵，使用 Seaborn 绘制热力图 (Heatmap)。行是 Site，列是 Customer，颜色深浅代表分配量。

要求：确保可视化代码包含 `plt.show()`，配色使用学术风格 (如 'viridis' 或 'Blues')。
```

---

## 🚀 比赛实战操作演示 (Simulation)

假设比赛中您遇到了一个 **“疫苗配送指派问题”**。

1.  **Notebook Cell 1**: 粘贴 `IP_Solver_Capsule` 类代码。运行。
2.  **Notebook Cell 2**:
    * **动作**：打开 `Prompt_0_Init.txt`，不做修改，发给 AI。
    * **AI回复**：系统就绪...
3.  **Notebook Cell 3**:
    * **动作**：打开 `Prompt_1_Variables.txt`。
    * **填空**：数据是 `vaccine.csv`；行是 Hospitals，列是 Communities；变量是 `x_ij` (0-1指派)。
    * **执行**：发送 -> 复制 AI 代码 -> 运行。
4.  **Notebook Cell 4**:
    * **动作**：打开 `Prompt_2_Constraints.txt`。
    * **填空**：约束A：每个社区必须被覆盖；约束B：医院产能有限；约束C：无（此题无 Big-M）。
    * **执行**：发送 -> 复制 AI 代码 -> 运行。
    * **观察**：屏幕输出 ✅ 审计通过。
5.  **Notebook Cell 5**:
    * **动作**：打开 `Prompt_3_Solve.txt`。
    * **填空**：最小化 `sum(x * distance)`。
    * **执行**：发送 -> 复制 AI 代码 -> 运行。
    * **观察**：屏幕输出 💎 目标函数值: 1204.5。
6.  **Notebook Cell 6**:
    * **动作**：打开 `Prompt_4_Delivery.txt`。
    * **填空**：画一个“社区-医院”匹配关系图。
    * **执行**：发送 -> 复制 AI 代码 -> 运行。
    * **结果**：得到 Excel 结果表、影子价格分析图、指派热力图。

> 这种**“分段结算”**的模式，即使中间某一步（比如约束写错了）出了问题，您只需要修改那一步的 Prompt 重新生成，而不需要推倒重来，容错率极高。